## Problem Set 4 - GAN and RNN
Designed by Xide Xia, Kubra Cilingir, Vijay Thakkar, Ali Siahkamari, and Brian Kulis, with help from Kun He.

This assignment will introduce you to:

Part 1 - GAN:
1. Implement your GAN on CelebA face data
2. Apply your GAN to Image Completion task

Part 2 - RNN:
1. Implementation of RNN and LSTM
2. Training your implemented LSTM
3. Composing music using LSTM


# Part 1: Training a GAN model on Celebrity dataset

GANs consists of two neural networks, a discriminator and a generator that are pitting against each other. If you do not feel comfortable with how they work, you are encouraged to check the original paper:
[Generative adversarial networks paper](https://arxiv.org/pdf/1406.2661.pdf)

<img src="gans.png" style="width: 600px;"/>

For a given real data coming from $ \mathbb  P_r$ and generator input distribution coming from $ \mathbb  P_z$, the original GAN objective function is defined as:

$$\mathcal{L} =\min_w \max_\phi \mathcal{l} \ (w, \phi) = \min_w \max_\phi \mathop{\mathbb{E}_{z \sim \mathbb  P_z}} [ \log (1 - f(g(z;w); \phi))] + \mathop{\mathbb{E}_{x \sim \mathbb  P_r}} [\log f(x; \phi)] \ \ \ \ \ \text{(1)} $$ 

where w and $\phi$ are the weights of the generator and the discriminator respectively.

However, empirically, if the generator's loss is converted to $-E_{z \sim p_z}[\log f(g(z;w), \phi)]$, then the gradient vanishing problem becomes less severe and training performance improves. The idea is to continue to use cross entropy minimization, but to get better gradients. **In this part, you need to use this modified loss to train GAN model.**

## Q1 Implement Your GAN
In this question, you will train a GAN to generate new face images by using celebrity dataset ([CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)) as the training set. You can find the data on scc:/projectnb/dl-course/HW4/celebA. We skipped the last 9 images in order to use as test images in Q3, but feel free to choose different face images!  

You need to write codes step by step as desired. 
- This time PIL image library is used to preprocess input data.
- The parameters are stored in an Arguments class instance. 
- Dataset class contains necessary functions for preprocessing and visualizing images.
- Generator and Discriminator models are defined as a function. 

Tricks that are used to improve training of the model:
- Deconvolutional layers
- Leaky RELU
- Using strided convolutions instead of pooling.
- Using batch normalization
- Not including fully connected layers


In [4]:
from __future__ import print_function
import os
import glob

import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
class Arguments(object):
    data_path = 'path/to/celebA'
    image_size = 64
    num_images = 202590
    batch_size = 64   
    dim_z = 100           
    n_gfilters = 64             
    n_ffilters = 64              
    n_epoch = 25            
    lr = 0.002           
    beta1 = 0.5   
    beta2 = 0.99


args = Arguments()

In [35]:
class Dataset(object): 
    
    def __init__(self, data_path, target_imgsize, num_imgs): 
        self.data_path = data_path
        self.num_imgs = num_imgs
        self.target_imgsize = target_imgsize 
        self.data_namelist = self.get_imagelist(self.data_path)


    def get_imagelist(self, data_path): 
        imgs_path = os.path.join(data_path, 'img_align_celeba/*.jpg')
        all_namelist = glob.glob(imgs_path, recursive=True)
        data_namelist = all_namelist[:self.num_imgs] 
        return data_namelist 

    def load_and_process_image(self, image_path):
        image = Image.open(image_path)
        j = (image.size[0] - 100) // 2
        i = (image.size[1] - 100) // 2
        image = image.crop([j, i, j + 100, i + 100])    
        image = image.resize([self.target_imgsize, self.target_imgsize], Image.BILINEAR)
        image = np.array(image.convert('RGB')).astype(np.float32)
        image = (image / np.amax(image) - 0.5) / 0.5
        return image
    
    def get_nextbatch(self, batch_size): 
        cur_idx = 0
        assert (batch_size > 0),"Give a valid batch size"
        while cur_idx + batch_size <= self.num_imgs:
            cur_namelist = self.data_namelist[cur_idx:cur_idx + batch_size]
            cur_batch = [self.load_and_process_image(image_path) for image_path in cur_namelist]
            cur_batch = np.array(cur_batch).astype(np.float32)
            cur_idx += batch_size
            yield cur_batch
      
    def show_image(self, image, normalized=True): 
        if not type(image).__module__ == np.__name__:
            image = image.numpy()
        npimg = (image * 0.5) + 0.5
        npimg.astype(np.uint8)
        plt.imshow(npimg, interpolation='nearest')

In [42]:
def generator(x, args, reuse=False):
    with tf.device('/gpu:0'):
        with tf.variable_scope("generator", reuse=reuse): 
            #Layer Block 1
            with tf.variable_scope("layer1"):
                deconv1 = tf.layers.conv2d_transpose(inputs=x, 
                                             filters=64*8, 
                                             kernel_size=4, 
                                             strides=1,
                                             padding='valid',
                                             use_bias=False,
                                             name='deconv')
                batch_norm1=tf.layers.batch_normalization(deconv1,
                                             name = 'batch_norm')
                relu1 = tf.nn.relu(batch_norm1, name='relu')
            #Layer Block 2
            with tf.variable_scope("layer2"):
                deconv2 = tf.layers.conv2d_transpose(inputs=relu1, 
                                             filters=64*4, 
                                             kernel_size=4,
                                             strides=2,
                                             padding='same', 
                                             use_bias=False,
                                             name='deconv')
                batch_norm2 = tf.layers.batch_normalization(deconv2,
                                             name = 'batch_norm')
                relu2 = tf.nn.relu(batch_norm2, name='relu')
            #Layer Block 3
            with tf.variable_scope("layer3"):
                deconv3 = tf.layers.conv2d_transpose(inputs=relu2, 
                                             filters=64*2, 
                                             kernel_size=4, 
                                             strides=2, 
                                             padding='same',
                                             use_bias = False,
                                             name='deconv')
                batch_norm3 = tf.layers.batch_normalization(deconv3, 
                                             name = 'batch_norm')
                relu3 = tf.nn.relu(batch_norm3, name='relu')
            #Layer Block 4
            with tf.variable_scope("layer4"):
                deconv4 = tf.layers.conv2d_transpose(inputs=relu3, 
                                             filters=64, 
                                             kernel_size=4, 
                                             strides=2,
                                             padding='same',
                                             use_bias=False,
                                             name='deconv')
                batch_norm4 = tf.layers.batch_normalization(deconv4,
                                             name = 'batch_norm')
                relu4 = tf.nn.relu(batch_norm4, name='relu')
            #Output Layer
            with tf.variable_scope("last_layer"):
                logit = tf.layers.conv2d_transpose(inputs=relu4, 
                                             filters=3, 
                                             kernel_size=4, 
                                             strides=2, 
                                             padding='same',
                                             use_bias=False,
                                             name='logit')
                output = tf.nn.tanh(logit) 
    return output, logit


In [41]:
def discriminator(x, args, reuse=False):
    with tf.device('/gpu:0'):
        with tf.variable_scope("discriminator", reuse=reuse): 
            with tf.variable_scope("layer1"):
                conv1 = tf.layers.conv2d(inputs=x,
                                         filters=64,
                                         kernel_size=4,
                                         strides=2,
                                         padding='same',
                                         use_bias=False,
                                         name='conv')
                relu1 = tf.nn.leaky_relu(conv1, alpha=0.2, name='relu')
            with tf.variable_scope("layer2"):
                conv2 = tf.layers.conv2d(inputs=relu1,
                                         filters=64*2,
                                         kernel_size=4,
                                         strides=2,
                                         padding='same',
                                         use_bias=False,
                                         name='conv')
                batch_norm2 = tf.layers.batch_normalization(conv2,name='batch_norm')
                relu2 = tf.nn.leaky_relu(batch_norm2, alpha=0.2, name='relu')
            with tf.variable_scope("layer3"):
                conv3 = tf.layers.conv2d(inputs=relu2,
                                         filters=64*4,
                                         kernel_size=4,
                                         strides=2,
                                         padding='same',
                                         use_bias=False,
                                         name='conv')
                batch_norm3 = tf.layers.batch_normalization(conv3, name='batch_norm')
                relu3 = tf.nn.leaky_relu(batch_norm3, name='relu')
            with tf.variable_scope("layer4"):
                conv4 = tf.layers.conv2d(inputs=relu3,
                                         filters=64*8,
                                         kernel_size=4,
                                         strides=2,
                                         padding='same',
                                         use_bias=False,
                                         name='conv')
                batch_norm4 = tf.layers.batch_normalization(conv4, name='batch_norm')
                relu4 = tf.nn.leaky_relu(batch_norm4, alpha=0.2, name='relu')
            with tf.variable_scope("last_layer"):
                logit = tf.layers.conv2d(inputs=relu4,
                                         filters=1,
                                         kernel_size=4,
                                         strides=1,
                                         padding='valid',
                                         use_bias=False,
                                         name='conv')
                output = tf.nn.sigmoid(logit) 
    return output

### Q1.1
Write a function that samples from a normal distribution with mean 0 and variance 1. The output vector should have dimensions (number of batch, 1, 1, dimension of the generator input).

In [43]:
def sample_z(z_dim, num_batch):
    pass
    return samples

### Q1.2
Write a function that takes discriminator outputs of real data and fake data and returns generator and discriminator losses.

In [ ]:
def get_losses(): #put inputs yourself 
    pass
    return d_loss, g_loss

### Q1.3
Write a function that takes optimization parameters as input and returns optimizer functions for discriminator and generator in tensorflow 

In [45]:
def get_optimizers(lr, beta1, beta2):
    pass
    return d_optimizer, g_optimizer

### Q1.4
Write a function that takes optimizers and losses of G and D, minimizes each loss and updates network parameters. Make sure to avoid training of G's parameters while D is being optimized.

In [ ]:
def optimize(d_optimizer, g_optimizer, d_loss, g_loss):
    pass
    return d_step, g_step

### Q1.5
Now, you will combine the functions you defined and perform a complete GAN training. Save generator's and dicriminator's loss function plots and test generator output images(You can write a helper function to put images in grid and display multiple images on the same plot). Save the weights of your model since they will be used in the next part.

In [48]:
def train(args):
    #fill
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epoch):
            for itr, real_batch in enumerate(get_nextbatch(batch_size)):
                #fill
                pass

### Q1.6
Train the model for 5 different learning rates, and embed loss functions on one plot for the generator losses and one plot for the discriminator losses. You can smooth graphs to make the plots more interpretable. Compare output images.

### Q1.7 (Bonus)
Search for different loss functions to improve GAN performance other than the ones used here, and apply one of them that ideally improves the training. You may need to makes changes in discriminator or generator model, if you do, redefine them here with different names. 

In [49]:
def get_new_losses(d_real_out, d_fake_out): 
    pass
    return d_loss, g_loss

def new_train(args):
    #fill
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epoch):
            for itr, real_batch in enumerate(get_nextbatch(batch_size)):
                #fill
                pass            

## Q2: Image Filling by using GAN model

In this part, you will implement an image filling application by using the GAN model you trained in the previous part. The main idea is that if a GAN model is trained well enough, then it should have a good representation of real data. For example, an image that is not from real data should not have a latent encoding on the z domain. Therefore, we want to find an encoding z that will be decoded by generator to be the closest to the occluded image. An example occluded input image is:  

<img src="image.png" style="width: 300px;"/>

We will search over z domain of the GAN model we trained in part 2. Write the codes step by step, as explained below: (observe that we will not be retraining GAN model!)

In [ ]:
#parameters
args.z_iter = 1500
args.alpha = 3e-3 
args.epsilon = 1e-12 
args.batch_size = 128
args.mask_height = 32
args.mask_width = 32

### Q2.1
First, create an image that looks normal, but has a 'superimposed' black square region. Basically you will take an image from CelebA dataset and remove a square region. 

Write a function that creates a mask which has the same size with the original input image; that has value 1 on a small square, and has value 0 on the other regions. Don't forget that we work on RGB scale!

In [ ]:
def create_mask(mask_height, mask_width, image_height, image_width):
    pass
    return mask

### Q2.2

Now get a sample image from CelebA dataset. Write a function that applies a mask to the image in a such way that output image should have value 1 on the missing patch and original pixel values on the other regions. Visualize the input image.

In [50]:
def apply_mask(mask, input_image):
    pass
    return masked_image

### Q2.3

We already got the idea that we want to search over the z domain to get a close complete image. How do we define closeness between images? We will combine two different losses for our task. 

- The first one is to ensure that we get realistic images. We will use discriminator GAN loss for that purpose. It favors z values that produces images that are classified as real by discriminator(closer to 1). $\alpha$ is a weighting term to balance between two losses and epsilon is for regularization:

$$ \mathcal L_{disc} = \alpha \log (1 - f(g(z)) + \epsilon)$$

- The second one is related to the context of the image. We do not want already existing pixels in the image to change much. So we use L2 loss to capture existing information. 
$$  \mathcal L_{context} = {\parallel (1 - M) \odot (y - g(z)) \parallel}_1 $$
where y is the occluded image, M is the mask defined above and z is the current input to the generator.

This time figure out the necessary inputs to the get_loss function by yourself!

In [ ]:
def get_loss(): #put inputs yourself
    pass
    return loss

### Q2.4

Now, write the main function by using the functions you wrote above. After each iteration, we clip z values between -1 and 1 to ensure that we are in the feasible latent space.
When the training is complete, you get a good z for the task, so giving it to the generator as an input will output an image. Fill the missing part of the input image with the output image by taking the corresponding patch.

In [ ]:
def image_filling(image, mask, args):
    #fill
    for itr in range(): 
        with tf.Session() as sess:
            #fill
            if itr % 10 == 0:
                #test
                pass

## Q3 Deep Understanding the GAN Loss (Bonus)

Assume that the generator is defined by a function $g(z ; w)$ where z is sampled from a random distribution $\mathbb P_z (z)$, and the discriminator is defined by $f(y ;\phi)$ _that tries to output high probability value if y is a real sample coming from the distribution $\mathbb P_r(x)$, and a low probability value if y is a fake sample, i.e. when $y= g(z; w)$_; whereas $w$ and $\phi$ are the weights of the generator and coming from the discriminator respectively. We ignore the dependence of weights to the data in notation.
Then the original objective function is defined as:

$$\mathcal{L}(\mathbb P_z, \mathbb P_r) =\min_w \max_\phi \mathcal{l} \ (\mathbb P_z, \mathbb P_r, w, \phi) = \min_w \max_\phi \mathop{\mathbb{E}_{z \sim \mathbb  P_z}} [ \log (1 - f(g(z;w); \phi))] + \mathop{\mathbb{E}_{x \sim \mathbb  P_r}} [\log f(x; \phi)] \ \ \ \ \ \text{(1)} $$ 

We will continue with notations $\mathcal{L}$ and $l(w, \phi)$ by dropping $\mathbb P_z$ and $\mathbb P_r$ from the notations.

### Q3.1.1
Define the distribution of the generator output as $\mathbb P_y$. Show that $\mathop{\mathbb{E}_{z \sim \mathbb  P_z}} [ \log (1 - f(g(z;w); \phi))] = \mathop{\mathbb{E}_{y \sim \mathbb  P_y}} [ \log (1 - f(y; \phi))]$. Is g invertible? How does this affect the method you use? Here is a useful link: [Change of variables](https://www.le.ac.uk/users/dsgp1/COURSES/LEISTATS/STATSLIDE4.pdf?fbclid=IwAR0XH32hlr58mJe80CPLS0_MdQ6dHtPaMsWFfEZDII37M5U4Mr0-6Zf6o_g)



### Q3.1.2

Rewrite this loss as a combination of (expected) binary cross entropy loss functions; in terms of $f$, $g$, $\mathbb P_r$ and $\mathbb P_y$. Relate it with the above statement (written in italic ) with the discriminator's aim, and conclude that the discriminator is acting as a classifier.

### Q3.2

Rewrite this loss as a combination of (expected) binary cross entropy loss functions; in terms of $f$, $g$, $\mathbb P_r$ and $\mathbb P_y$. Relate it with the above statement (written in italic ) with the discriminator's aim, and conclude that the discriminator is acting as a classifier.
Now let us interpret this objective function from another but related perspective. This time, we will show that an **optimal** discriminator measures [Jensen-Shannon](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence) divergence between real and fake distributions. Do the following steps:

### Q3.2.1

Write the **optimal** discriminator function $f^*$ in terms of $\mathbb  P_r$ and $\mathbb  P_y$ (hint: take proper derivatives to reach the solution. You may want to define the discriminator output as a new variable).



### Q3.2.2

Now put $f^*$ found in 1.2.1 into the equation and find $ \max_\phi$. Use the equality below to conclude  

$$ \max_\phi l(w, \phi) := \mathcal{l}(w, \phi_{f^*}) = 2 * JS(\mathbb  P_r \parallel \mathbb  P_y ) - log(4) $$

Briefly mention the difference between JS and KL divergences.

(Note: add and substract $\log 4$ to get to the solution. First try to write in terms of [KL divergences](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence), then use the relation below to achieve JS divergence) 


$$JS(p, q) = \frac{KL(p \parallel \frac{p+q}{2}) + KL(q \parallel \frac{p+q}{2})}{2}$$


$$KL(p,q) = \int p \log \frac{p}{q} d\mu$$ 

1.2.3 Now the generator's loss function becomes: $\min_w \mathcal{l}(w, \phi_{f^*})$. Since JS divergence has minimum 0 we reach the global optimum at the value $- \log 4$. Show that this optimum is achieved when  $\mathbb  P_r = \mathbb  P_y$ and it is unique. Comment on it.

### Q3.3

Now consider the problem as an optimization problem with the gradient descent method. In reality, with this loss function, instead of training the discriminator first and getting $f^*$, the training is done alternatingly. We will create a simple scenario where we start with an optimal f, and training g would not work. 

### Q3.3.1

Assume real data is of the form $(0, x)$ and the generator input is of the form $(1, x)$, and $x \sim Unif(0,5)$ The generator is defined by a single parameter $w$, which adds $(w, 0)$ to the input. Observe that the generator has enough capacity to capture real data.  

Write an experiment that starts from any  $w \neq w_{optimal}$. Try to apply gradient descent on the generator with the optimal discriminator. Conclude that an optimal solution cannot be found.(Consider the gradients of the loss and $w$ update of the generator. Also remember that an optimal discriminator measure JS divergence!)




### Q3.3.2
In addition, we do not have a real data distribution in a real case, so the loss function turns out to be a finite sum. What might go wrong because of this approximation?

# Part II: RNN 

## Q4. RNN Example

In this example we train an RNN to infer the parameters of a simple dynamical system.
First, we simulate a dynamical system with known parameters (random numbers), and use it to generate outputs. Then, we train an RNN on the generated datapoints, attempting to infer the original parameters.
You are expected to run and study the provided code, which will be helpful for the second part (implementing LSTM).

1. We define a discretized [dynamical system](https://en.wikipedia.org/wiki/Dynamical_system).
At each discrete time $t$, the system observes input $x_t$. The system maintains some "state" $h_t$, which will be updated over time.
Specifically, the states are updated by the following rule: $h_t = \max(0, 1-(Wx_t+h_{t-1}))$, where $W$ is a parameter matrix.
2. In this example, the input data $\{x_t\}$ is randomly generated, and the weight matrix $W$ is also drawn randomly. The system starts from an initial hidden state $h_0$, and runs for $t=1,\ldots,T$.
3. Given the sequence of states $\{h_1,\ldots,h_T\}$, we would like to infer the weights $W$.

To start with, the following code segment generates and displays the data.
Refer to `data_generator.py` for details of data generation.

In [ ]:
# Here we present sample data generated 
import numpy as np
import data_generator
import matplotlib.pyplot as plt

(h_0, w), x, h = data_generator._build_rnn_testdata_matrix()

print('variable name, shape, min, max: ')
for v, name in zip([h_0, w, x], ['h0', 'w', 'x']):
    print(name, v.shape, np.min(v), np.max(v))
norm_x_t = np.sum(x**2, axis=1)
plt.title('||x(t)||')
plt.plot(np.arange(norm_x_t.shape[0]), norm_x_t)
plt.show()

We model the dynamical system with an RNN. We would like the state generated by the RNN to match the actual observed, and we use L2 loss for this purpose.
This RNN is a *regression* model since it outputs real values.

Below, `build_rnn_regression_model()` gives the model definition, and `train_rnn_with_noise()` generates a batch of data and then runs training on it. 
We corrupt the data generation process with noise, and let the dimensionality of the state $h$ be a free parameter. Therefore, the training function takes two input arguments: `noise_level` and `n_hidden_dim`.   You will later see how varying them affects reconstruction quality.

In [ ]:
import datetime
import os
import numpy as np
import tensorflow as tf
from __future__ import print_function

#############################################################################
# RNN model graph
def build_rnn_regression_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_hidden_dim, n_input_dim
    with tf.Graph().as_default() as g:
        # inputs to the dynamical system
        X = tf.placeholder(tf.float32,
                           [None, shape['n_steps_per_batch'], shape['n_input_dim']])
        # observed state from the dynamical system
        y = tf.placeholder(tf.float32, [None, shape['n_hidden_dim']])
        
        with tf.variable_scope('weights'):
            # weight matrix
            w = tf.get_variable('w', [shape['n_input_dim'], shape['n_hidden_dim']])
            # initial state
            h_0 = tf.get_variable('h_0', [shape['n_hidden_dim']])
            
        # for t = 1 to T, update state 
        h_t = h_0
        for t in range(shape['n_steps_per_batch']):
            x_t = X[:, t, :]
            h_t = tf.maximum(0.0, 1 - (tf.matmul(x_t, w) + h_t))
        
        # loss: L2
        loss = tf.nn.l2_loss(h_t - y, name='loss')
        train_op = tf.train.AdamOptimizer(0.1).minimize(loss)
        summ = tf.summary.scalar('loss_sum_%dd' % shape['n_hidden_dim'], loss)
        
    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'weights': {'w': w, 'h_0': h_0}, 'graph': g}

#############################################################################
# Main train loop for an RNN regression model
# 
# This takes synthetic data generated by data_generator.build_dataset()
# the weight matrix W is then inferred with back-prop 
def train_rnn_with_noise(noise_level, n_hidden_dim):
    # generate data
    shapes = dict(n_hidden_dim=n_hidden_dim, n_input_dim=15, n_steps_per_batch=100)
    rnn_dataset = data_generator.build_dataset('rnn', noise=noise_level, **shapes)
    (h0_true, w_true), batched_data = rnn_dataset  # "true" weights
    # build RNN model
    model = build_rnn_regression_model(shapes)
    
    #logdir = './tensorboard/rnn_demo'  # if on Windows
    logdir = '/tmp/tensorboard/rnn_demo'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    # If you want to see the plots, run tensorboard:
    # $ tensorboard --logdir=[your_logdir]
    #
    # If you use SCC, you can forward the 6006 port from the cluster 
    # to your local machine via:
    # $ ssh [SCC_cluster_name] -L 6006:localhost:6006
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d_noise=%.2f' % (n_hidden_dim, noise_level)
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    max_iter_i = 0
    with model['graph'].as_default() as g, tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(50):
            loss_val, w_dist, h0_dist, iter_i = None, None, None, None
            for iter_i, data_batch in enumerate(batched_data):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                # run training step
                train_feed_dict = dict(zip(model['inputs'], data_batch))
                to_compute = [model['train_op'], model['summ'], model['loss'],
                              model['weights']['w'], model['weights']['h_0']]
                _, summ, loss_val, w_val, h0_val = sess.run(to_compute, train_feed_dict)
                
                # compute reconstruction error
                w_err = np.linalg.norm(w_true-w_val)
                h0_err = np.linalg.norm(h0_true-h0_val)
                
                # for tensorboard
                sum_writer.add_summary(summ, global_step)
                sum_writer.add_summary(tf.Summary(value=[
                    tf.Summary.Value(tag="w_true_dist_%dd" % n_hidden_dim,
                                     simple_value=w_err),
                ]), global_step)
                sum_writer.add_summary(tf.Summary(value=[
                    tf.Summary.Value(tag="h_true_dist_%dd" % n_hidden_dim,
                                     simple_value=h0_err),
                ]), global_step)
                sum_writer.flush()
                
            print('epoch %d, loss=%g, w_err=%g'%(epoch_i, loss_val, w_err))
            if global_step > 200: 
                break  # just train for 200 iterations

Now test the RNN wth varying noise levels and hidden dimensionalities.
- For each combination of `n_hidden_dim` and `noise_level`, report the reconstruction error (`w_err`).
- Describe how the hidden dimentionality and the noise level influence reconstruction quality. And briefly explain why.

In [ ]:
# Experimenting with different data noise levels and hidden dimentionalities
# We are lucky to know the true hidden dimentionality in our simultaion
for n_hidden_dim in [10, 100, 1000]:
    for noise_level in [0, 0.1, 0.5]:
        print(n_hidden_dim, noise_level)
        train_rnn_with_noise(noise_level, n_hidden_dim)

## Q5: LSTM implementation

Now let's attempt to recover the weights in dynamical system simulated with an LSTM.  Although LSTMs are already implemented in TensorFlow ([tutorial here](https://www.tensorflow.org/tutorials/recurrent)) ([source here](https://github.com/tensorflow/tensorflow/blob/efe5376f3dec8fcc2bf3299a4ff4df6ad3591c88/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py#L264)), you will be implementing a simple LSTM from scratch using Tensorflow in this part. 


### Q5.1 Implement an LSTM 
Implement an LSTM model in an analogous way,  in functions `build_lstm_regression_model()` and `train_lstm_with_noise()` below.
The RNN regression implementation above should give you some ideas.
We already provided an LSTM version of the dynamical system generator in the dataset generator code.

- Specifically, you can follow and implement Eq. 1-6 from [this link](http://deeplearning.net/tutorial/lstm.html) in `build_lstm_regression_model()`. 
You may simplify your code by concatenating $x$ and $h$.
- Afterwards, implement `train_lstm_with_noise()` to train the LSTM and recover the parameters. Compute the reconstruction errors for $W_c$ and $U_c$, which are the parameters used in Eq. 2 in the link.
- For each combination of hidden dimension and noise level, report the reconstruction error (`w_err`, `u_err`) you get from the LSTM.

Note:
- The weights might not get reconstructed correctly in the LSTM case even without noise. (Why?)
Nevertheless, the loss should decrease.

In [ ]:
def build_lstm_regression_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_hidden_dim, n_input_dim
    with tf.Graph().as_default() as g:
        # inputs
        X = tf.placeholder(tf.float32,
                           [None, shape['n_steps_per_batch'], shape['n_input_dim']])
        # observed outputs
        y = tf.placeholder(tf.float32, [None, shape['n_hidden_dim']])
        
        #################################################################
        ####################  PUT YOUR CODE HERE   ######################
        # define the parameters in the LSTM (scope: weights)
        # and carry out the computation according to Eq. 1-6 in the link
        with tf.variable_scope('weights'):
            pass
        output = None  # put your result in variable 'output'
        #################################################################
        
        # loss and train_op
        loss = tf.nn.l2_loss(output - y, name='loss')
        train_op = tf.train.AdamOptimizer(0.1).minimize(loss)
        summ = tf.summary.scalar('loss_sum_%dd' % shape['n_hidden_dim'], loss)

    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'weights': {'w_c': w_c, 'u_c': u_c}, 
            'graph': g}
    
    
def train_lstm_with_noise(noise_level, n_hidden_dim):
    # generate data and random weights
    shapes = dict(n_hidden_dim=n_hidden_dim, n_input_dim=15, n_steps_per_batch=100)
    weights, batched_data = data_generator.build_dataset('lstm', noise=noise_level, **shapes)
    w_c, u_c = weights[3], weights[7]  # the "true" weights to recover: Wc & Uc (in Eq.2)
    
    # this is the function you implemented
    model = build_lstm_regression_model(shapes)
    
    #logdir = './tensorboard/lstm_demo'  # if on Windows
    logdir = '/tmp/tensorboard/lstm_demo'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d_noise=%.2f' % (n_hidden_dim, noise_level)
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    
    max_iter_i = 0
    with model['graph'].as_default() as g, tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(10):  # 10 epochs by default, feel free to change
            loss_val, w_err, u_err, iter_i = None, None, None, None
            for iter_i, data_batch in enumerate(batched_data):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                ###############################################################
                ###################   PUT YOUR CODE HERE   ####################
                train_feed_dict = None  # define your train_feed_dict
                to_compute = []  # the things to compute, including at least:
                                 # loss_val, w_err, u_err
                outputs = sess.run(to_compute, train_feed_dict)
                w_err = None  # compute them and report
                u_err = None
                # [optional] you can also do early stopping, e.g. 300 iterations
                # if global_step > 300:
                #    break
                # use sum_writer for tensorboard, see train_rnn_with_noise()
                sum_writer.add_summary(...)
                sum_writer.flush()
                ###############################################################

In [ ]:
for n_hidden_dim in [10, 100, 1000]:
    for noise_level in [0, 0.1, 0.5]:
        print(n_hidden_dim, noise_level)
        train_lstm_with_noise(noise_level, n_hidden_dim)

## Q6: Compose Music using a LSTM 

In this part you will train a music prediction LSTM that predicts the next note given previous notes.  

### MIDI Data

A file with the .MID or .MIDI file extension is a Musical Instrument Digital Interface file. Unlike regular audio files like MP3 or WAV files, MIDI files don't contain actual audio data and are therefore much smaller in size. For example, MID files explain what notes are played and how long or loud each note should be. Instead, they are basically instructional files that explain how the sound should be produced once attached to a playback device or loaded into a particular software program that knows how to interpret the data. This makes MIDI files perfect for sharing musical information between similar applications. Learn how to play a midi file: https://www.lifewire.com/midi-file-2621979.

In this problem, we will use a small Mozart MIDI Corpus dataset [Mozarella](https://github.com/thakkarV/Mozarella) to train a LSTM for composing music. For each midi file, we extract an embedding matix with size of [num_note,87] - it has num_note continous notes and each note is represented by a 87-dim one-hot vector. Besides of embedding matix, we also prepare a matrix of duration with size of [num_note, 1] to save the duration time for each note. In this problem, we assume each note has same duration. The embedding and duration matrix are save on scc:  /projectnb/dl-course/HW4/Mozarella/midi_embedding.

We have installed mozarella and midi on scc for you. If you want to play on a local machine, please follow the instruction of installation on [Mozarella](https://github.com/thakkarV/Mozarella).




### Q6.1 LSTM with Discrete Outputs
Compared to the LSTM you implemented in the previous part, the main difference in the music prediction LSTM is that it predicts *discrete* outputs, therefore it is a classification model. As you may recall, the usual choice of loss function for classification is softmax + cross entropy.

We have already provided functions for loading the training data. Please define your discrete LSTM in `build_lstm_discrete_prediction_model()`.

Hints: 
- Your LSTM should predict a single note at a time. We are not considering  many-to-many LSTMs here.
- Feeding the previous input into the LSTM should help.

Feel free to change the paramenters or codes if needed. 

In [ ]:
import midi_reader
emb_pkl_path = '/projectnb/dl-course/HW4/Mozarella/midi_embedding/'


def get_default_gpu_session(fraction=0.333):
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    return tf.Session(config=config)

def run_midi_train(n_hid):
    # generate data
    btg = midi_reader.batch_midi_generator_from_pkl(emb_pkl_path, batch_size=15, seq_size=360)
    shape = dict(n_steps_per_batch=360, n_unique_ids=87, n_hidden_dim=n_hid)
    # define LSTM
    model = build_lstm_discrete_prediction_model(shape)
    
    #logdir = './tensorboard/midi'  # if on Windows
    logdir = '/tmp/tensorboard/midi'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d' % n_hid
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    max_iter_i = 0
    with model['graph'].as_default() as g, get_default_gpu_session(0.9) as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(10):
            for iter_i, data_batch in enumerate(btg):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                # run training step
                train_feed_dict = dict(zip(model['inputs'], data_batch))
                to_compute = [model['train_op'], model['summ'], model['loss']]
                _, summ, loss_val = sess.run(to_compute, train_feed_dict)
                
                # for tensorboard
                sum_writer.add_summary(summ, global_step)
                sum_writer.flush()
                
            print(loss_val, end=', ')

            # test generation
            pred_length = 50
            data_input = next(iter(btg))[0][[0]]
            original_sample = data_input.copy()
            pred_seq = []
            for _ in range(pred_length):
                pred = sess.run(model['pred'], {model['inputs'][0]: data_input})
                pred_seq.append(pred[0])
                data_input = np.roll(data_input, -1, axis=1)
                data_input[0, -1] = pred[0]

            print(pred_seq)
            print()
            
                
def build_lstm_discrete_prediction_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_unique_ids, n_hidden_dim
    with tf.Graph().as_default() as g:
        X = tf.placeholder(tf.int64, [None, shape['n_steps_per_batch']])
        y = tf.placeholder(tf.int64, [None])
        
        ################################################################
        ####################   PUT YOUR CODE HERE   ####################
        # define LSTM parameters (scope: weights)
        with tf.variable_scope('weights'):
            pass
        logits = None  # compute logits for each discrete output
        loss = None  # compute loss with respect to (logits, y)
        ################################################################

        # pred, train_op
        pred = tf.argmax(logits, axis=1)
        train_op = tf.train.AdamOptimizer().minimize(loss)
        summ = tf.summary.scalar('loss_summ', loss)

    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'graph': g, 'pred': pred}

In [ ]:
hidden = 50
run_midi_train(hidden)

### Q6.2 Let's Check Your Music!

Now you have your prediction and the next step to convert it back into midi files using midi_reader.save_to_mid(pred_seq, 'path/to/save/out.mid')

Please submit your result in .mid format. Report the performance and explain why. 

## Q7 Composing Better Music (Bonus)

Try to improve your music prediction.

Hint:

You can try to improve your music in different ways. For example:

1. Use a better RNN model such as bidirectional LSTM.
2. Add the duration of each note into your model. 

The duration of each note is save in all_raw_duration.pkl at scc: /projectnb/dl-course/HW4/Mozarella/midi_embedding. Or you can extract the features from midi data using mozarella.midi_embedding_generator(). The output of midi_embedding_generator() is a [num_note,88] embedding where the first 87 elements are one-hot embedding and the last one is the corresponding duration. You can use mozarella.generate_midi_from_embeddings() to save your results. 

Please submit your result in .mid format. Report the performance and explain why.


Congratulations!